# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,276.73,81,100,5.86,TF,1723516537
1,1,tofol,5.3248,163.0078,301.79,72,29,4.54,FM,1723516538
2,2,puerto ayora,-0.7393,-90.3518,293.46,94,86,3.58,EC,1723516274
3,3,edinburgh of the seven seas,-37.0676,-12.3116,284.82,67,33,4.13,SH,1723516542
4,4,akureyri,65.6835,-18.0878,283.11,87,100,8.23,IS,1723516543


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

# # Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 0.85,
    color = 'City'
)

# # Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Humidity'] <= 60)  & (city_data_df['Max Temp'] <= 300)] 

# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,kodiak,57.7900,-152.4072,289.88,42,20,6.17,US,1723516548
11,11,college,64.8569,-147.8028,289.30,53,75,5.14,US,1723516552
12,12,jesus maria,-30.9815,-64.0942,282.24,33,0,4.59,AR,1723516381
16,16,badger,64.8000,-147.5333,289.25,58,75,5.14,US,1723516559
25,25,blackmans bay,-43.0167,147.3167,291.33,52,99,1.34,AU,1723516569
...,...,...,...,...,...,...,...,...,...,...
511,511,derzhavinsk,51.1000,66.3167,295.19,47,95,7.86,KZ,1723517170
524,524,inverloch,-38.6333,145.7167,290.91,58,2,5.57,AU,1723517185
554,554,greymouth,-42.4667,171.2000,284.72,56,48,0.46,NZ,1723517223
556,556,sena madureira,-9.0656,-68.6569,295.94,50,19,1.30,BR,1723517225


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.loc[:,['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,kodiak,US,57.7900,-152.4072,42,
11,college,US,64.8569,-147.8028,53,
12,jesus maria,AR,-30.9815,-64.0942,33,
16,badger,US,64.8000,-147.5333,58,
25,blackmans bay,AU,-43.0167,147.3167,52,
...,...,...,...,...,...,...
511,derzhavinsk,KZ,51.1000,66.3167,47,
524,inverloch,AU,-38.6333,145.7167,58,
554,greymouth,NZ,-42.4667,171.2000,56,
556,sena madureira,BR,-9.0656,-68.6569,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
#filters = f'circle: {lat}, {lng}, {radius}'
params = {
    'limit' : limit,
    'categories' : 'accomidation.hotel',
    'apiKey' : geoapify_key,
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle: {lng},{lat},{radius}'
    params["bias"] = f'proximity: {lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kodiak - nearest hotel: No hotel found
college - nearest hotel: No hotel found
jesus maria - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
ilulissat - nearest hotel: No hotel found
mildura - nearest hotel: No hotel found
chelm slaski - nearest hotel: No hotel found
portland - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hotel found
ghanzi - nearest hotel: No hotel found
yuanping - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
maloye ganyushkino - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
hailar - nearest hotel: No hotel found
manhuacu - nearest hotel: No hotel found
urumqi - nearest hotel: No hotel found
podgornaya - nearest hotel: No hotel found
ulaangom - nearest hotel: No hotel found
truth or consequences - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
nuuk - nearest hotel: No hotel found
k

,City,Country,Lat,Lng,Humidity,Hotel Name
8,kodiak,US,57.7900,-152.4072,42,No hotel found
11,college,US,64.8569,-147.8028,53,No hotel found
12,jesus maria,AR,-30.9815,-64.0942,33,No hotel found
16,badger,US,64.8000,-147.5333,58,No hotel found
25,blackmans bay,AU,-43.0167,147.3167,52,No hotel found
...,...,...,...,...,...,...
511,derzhavinsk,KZ,51.1000,66.3167,47,No hotel found
524,inverloch,AU,-38.6333,145.7167,58,No hotel found
554,greymouth,NZ,-42.4667,171.2000,56,No hotel found
556,sena madureira,BR,-9.0656,-68.6569,50,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot


# Display the map
# YOUR CODE HERE